In [63]:
import csv
import math
csvfile = open('nycPercentSorted.csv', newline='')
csvAvg = open('nycFreqSorted.csv',newline='')

c = csv.reader(csvfile)
d = csv.reader(csvAvg)

average=[0] * 71  # change to higher value to make it non-redundant for now it is equal to the no of total unique crimes
i=0
yearsToTrain = 14    # change s value to no of years to be used for training 

for row in d:
    if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year' or row[0]=='month':
        continue
    else:
       j=0
       for month in range (1,(yearsToTrain*12)+1):
           if float(row[month]) !=0:    
               j+=1
               average[i] += float(row[month])    # maintain grand average for scale later 
       average[i] /= j
       i+=1
      
percentages = []
totals = []
categories =[]
i = 0
monYear = []

for row in c:
    
    if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year':
        continue
    elif row[0]=='month':
        for month in range(1,(yearsToTrain*12)+1):
            monYear.append(float(row[month]))
    else:
       temp = []
       categories.append(row[0])
       
       for month in range (1,(yearsToTrain*12)+1):
           if float(row[month]) !=0:    
               j+=1
           
           temp.append(float(row[month]))   # add monthly percentages of a particular crime to  list 
       
       percentages.append(temp)   # add monthly percentages list to a list 
       i += 1

predictions=[]


def predictOverYears(start,end):           # fn to give prediction for one year mentioned by start and end
    predictAyear = []
    for idx in range (0,i):
        crime = percentages[idx]
        sum =0
        index = -1
        for it in range (start,end):     # sum for caluclationg confidence later on             
            sum+=crime[it]
    
        predictMonth = []

        for it in range (start,end):
            avg = sum - crime[it]
            avg=float(avg/11)
            if avg != 0:
                likeness = ((float(crime[it])/(avg))) * 100    # measure confidence rate for that month for that crime
            else :
                likeness=0
            temp = [likeness,monYear[it]]
            predictMonth.append(temp)                   # append confidence rate and index of month predicted

        predictAyear.append(predictMonth)           # append all month's of that year likliness to a list 
    return predictAyear


for j in range(0,yearsToTrain):            
    n=j*12
    predictions.append(predictOverYears(n,n+12))

# print(predictions[1])

predictionPerCrime = []
for idx in range(0,i):
    tem = []
    for k in range(0,yearsToTrain):
        tem.append(predictions[k][idx])           # get each individual year predictions and add it to a new list 
    predictionPerCrime.append(tem)
    
# print(predictionPerCrime[0])



grandPrediction = []
for idx in range(0,i):
    perCrimeConf=[]
    for month in range(0,12):
        sumOverYears=0
        for yr in range(0,yearsToTrain):
            sumOverYears+=predictionPerCrime[idx][yr][month][0]
        tem=sumOverYears/yearsToTrain   # sum of confidence of a month across years and divide by yearsToTrain
        perCrimeConf.append(tem)
    grandPrediction.append(perCrimeConf)  # append confidence rates of each month per crime to list 



# print(grandPrediction[0][2])

for j in range(0,i):
    low=9999
    for m in range(0,12):
        if grandPrediction[j][m] < low:
            low = grandPrediction[j][m]
    if low != 0 :
         low = math.ceil(low) -2  # find lowest value ceil it and subtrat 2 from it
    for n in range(0,12):
        grandPrediction[j][n] -= low  # then subtract this val(low) from all

for j in range(0,i):
    max = 0
    avg = 0
    for m in range (0,12):
         avg+=grandPrediction[j][m]
         if grandPrediction[j][m] > max :
             max = grandPrediction[j][m]
    avg=avg/12
    ideal = ( max + avg)/2     # compute ideal confidence and acceptable confidence level
    print(categories[j],"has a ideal confidence:",ideal,"and acceptable confidence:",avg,"monthly predictons are:")
    for k in range(0,12):
        print("     Month",k+1,"confidence rate:",grandPrediction[j][k])   
             







PETIT LARCENY has a ideal confidence: 11.219324495540059 and acceptable confidence: 8.020255034564986 monthly predictons are:
     Month 1 confidence rate: 5.052643897461877
     Month 2 confidence rate: 1.0452191977851726
     Month 3 confidence rate: 1.1401784873615526
     Month 4 confidence rate: 4.056503737070244
     Month 5 confidence rate: 6.162814248270237
     Month 6 confidence rate: 9.224650631446622
     Month 7 confidence rate: 11.533316968107258
     Month 8 confidence rate: 12.497407858651627
     Month 9 confidence rate: 10.91798140071792
     Month 10 confidence rate: 9.890914622341114
     Month 11 confidence rate: 10.303035409051077
     Month 12 confidence rate: 14.418393956515132
HARRASSMENT 2 has a ideal confidence: 7.49688539613631 and acceptable confidence: 5.012887206810952 monthly predictons are:
     Month 1 confidence rate: 1.5256988881695008
     Month 2 confidence rate: 1.9170200569534046
     Month 3 confidence rate: 4.0693127829251665
     Month 4 confi